<a href="https://colab.research.google.com/github/zcongfly/huggingface-nlp-learning-note/blob/main/02_%E7%AE%A1%E9%81%93%E7%9A%84%E5%86%85%E9%83%A8_(PyTorch).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 管道的内部 (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

本章将从一个端到端的示例开始，在该示例中，我们一起使用模型和tokenizer分词器来复制Chapter 1中引入的函数pipeline(). 接下来，我们将讨论模型API：我们将深入研究模型和配置类，并向您展示如何加载模型以及如何将数值输入处理为输出预测。

然后我们来看看标记器API，它是pipeline()函数的另一个主要组件。它是作用分词器负责第一个和最后一个处理步骤，处理从文本到神经网络数字输入的转换，以及在需要时转换回文本。最后，我们将向您展示如何处理在一个准备好的批处理中通过一个模型发送多个句子的问题，然后详细介绍pipeline()函数。

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 99.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 133.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[{'label': 'POSITIVE', 'score': 0.9598049521446228},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

此管道将三个步骤组合在一起：预处理、通过模型传递输入和后处理：

![](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/full_nlp_pipeline.svg)

让我们快速浏览一下这些内容。

## 使用分词器进行预处理

与其他神经网络一样，Transformer模型无法直接处理原始文本， 因此我们管道的第一步是将文本输入转换为模型能够理解的数字。 为此，我们使用tokenizer(标记器)，负责：

* 将输入拆分为单词、子单词或符号（如标点符号），称为标记(token)
* 将每个标记(token)映射到一个整数
* 添加可能对模型有用的其他输入

所有这些预处理都需要以与模型预训练时完全相同的方式完成，因此我们首先需要从Model Hub中下载这些信息。为此，我们使用AutoTokenizer类及其from_pretrained()方法。使用我们模型的检查点名称，它将自动获取与模型的标记器相关联的数据，并对其进行缓存（因此只有在您第一次运行下面的代码时才会下载）。

因为sentiment-analysis（情绪分析）管道的默认检查点是distilbert-base-uncased-finetuned-sst-2-english（你可以看到它的模型卡[here](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english))，我们运行以下程序：

In [ ]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

一旦我们有了标记器，我们就可以直接将我们的句子传递给它，然后我们就会得到一本字典，它可以提供给我们的模型！剩下要做的唯一一件事就是将输入ID列表转换为张量。

要指定要返回的张量类型（PyTorch、TensorFlow或plain NumPy），我们使用return_tensors参数：

现在不要担心填充和截断；我们稍后会解释这些。这里要记住的主要事情是，您可以传递一个句子或一组句子，还可以指定要返回的张量类型（如果没有传递类型，您将得到一组列表）。

以下是PyTorch张量的结果：

In [ ]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


## 浏览模型

我们可以像使用标记器一样下载预训练模型。Transformers提供了一个AutoModel类，该类还具有from_pretrained()方法：

In [ ]:
from transformers import AutoModel

checkpoint="distilbert-base-uncased-finetuned-sst-2-english"
model=AutoModel.from_pretrained(checkpoint)

Some weights of the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing DistilBertModel: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


在这个代码片段中，我们下载了之前在管道中使用的相同检查点（它实际上应该已经被缓存），并用它实例化了一个模型。

这个架构只包含基本转换器模块：给定一些输入，它输出我们将调用的内容隐藏状态（hidden states），亦称特征（features）。对于每个模型输入，我们将检索一个高维向量，表示**Transformer模型对该输入的上下文理解**。

如果这不合理，不要担心。我们以后再解释。

虽然这些隐藏状态本身可能很有用，但它们通常是模型另一部分（称为头部(head)）的输入。 在Chapter 1中，可以使用相同的体系结构执行不同的任务，但这些任务中的每个任务都有一个与之关联的不同头。

### 高维向量？

Transformers模块的矢量输出通常较大。它通常有三个维度：

* Batch size: 一次处理的序列数（在我们的示例中为2）。
* Sequence length: 序列的数值表示的长度（在我们的示例中为16）。
* Hidden size: 每个模型输入的向量维度。

由于最后一个值，它被称为“高维”。隐藏的大小可能非常大（768通常用于较小的型号，而在较大的型号中，这可能达到3072或更大）。

如果我们将预处理的输入输入到模型中，我们可以看到这一点：

In [ ]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([2, 16, 768])


Transformers模型的输出与namedtuple或词典相似。您可以通过属性（就像我们所做的那样）或键（输出["last_hidden_state"]）访问元素，甚至可以通过索引访问元素，前提是您确切知道要查找的内容在哪里（outputs[0]）。

### **inputs

这行代码的作用是使用预训练的 DistilBERT 模型对输入进行推理，得到模型的输出。

首先，我们使用 `AutoModel.from_pretrained()` 函数从 Hugging Face 的模型库加载了一个名为 `distilbert-base-uncased-finetuned-sst-2-english` 的 DistilBERT 模型。这个模型是在 SST-2 数据集上进行微调的，并可以用于进行二分类任务。

接下来，我们将 `inputs` 变量作为参数传递给模型 `model`。由于 `inputs` 是一个字典对象，而我们在传递参数时需要将其展开成关键字参数形式，因此需要使用 `**inputs` 的形式将其展开。具体来说，这行代码的作用相当于执行了以下语句：

```python
outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
```

也就是将 `inputs` 字典中的 `input_ids` 和 `attention_mask` 两个键分别作为参数传递给模型。这样做的好处是可以直接将字典作为输入传递给模型，比多次手动传递参数更加方便。

最后，我们打印了模型的输出张量 `last_hidden_state` 的形状。这个张量的大小是 $2\times16\times768$，其中 $2$ 表示批量大小（即输入样本的数量），$16$ 表示序列长度（即每个样本的输入序列长度），$768$ 表示每个标记的隐藏状态向量维度。这个张量包含了每个输入标记的隐藏状态表示，可以作为下游任务的特征输入。

准确来说，只有在使用关键字参数传递参数时，才能使用 **字典 的形式将字典展开。如果是使用位置参数传递参数，是无法使用这种方式的。

如果使用 **字典 的形式传递参数，Python 会将字典中的所有键值对分别解包成关键字参数。因此，只有字典中的键（即参数名）与函数或方法的参数名一致，才能使用这种方式进行传参。

除了使用 **字典 的形式传递参数，还可以使用 *元组 的形式将元组展开成位置参数。类似地，只有元组的元素数量与函数或方法的参数个数一致，才能使用这种方式进行传参。

需要注意的是，使用 **字典 或 *元组 的展开方式虽然方便，但也有可能导致错误，特别是当参数名或参数顺序发生变化时。因此，在编写代码时需要仔细检查参数名和参数顺序是否正确。


### 模型头：数字的意义

模型头将隐藏状态的高维向量作为输入，并将其投影到不同的维度。它们通常由一个或几个线性层组成：

![](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/transformer_and_head.svg)

Transformers模型的输出直接发送到模型头进行处理。

在此图中，模型由其嵌入层和后续层表示。嵌入层将标记化输入中的每个输入ID转换为表示关联标记(token)的向量。后续层使用注意机制操纵这些向量，以生成句子的最终表示。

在Transformer的经典框架中，模型头是指注意力机制的一部分，用于在不同任务上进行特定的计算。模型头通常接收隐藏状态的高维向量作为输入，并对其进行一系列的变换操作。

具体地说，隐藏状态是Transformer模型中编码器和解码器各个层之间传递的信息。每个编码器和解码器层都会生成一个隐藏状态，该隐藏状态是一个包含多个向量的矩阵。模型头接收这个隐藏状态作为输入，并将其投影到不同的维度。投影通常通过一个或几个线性层来实现，这些线性层可以将输入的高维向量映射到其他维度上。

模型头的设计取决于具体的任务。例如，在机器翻译任务中，可以使用一个模型头来计算源语言和目标语言之间的注意力分布，以便在解码器中生成正确的翻译。在问答任务中，可以使用不同的模型头来预测答案的起始和结束位置。

因此，模型头是Transformer模型中用于执行特定任务的组件，它接收隐藏状态的高维向量作为输入，并通过投影到不同的维度来进行任务特定的计算。

#### 模型头vs多头注意力

模型头和多头注意力机制中的多头是相互关联的概念。

在Transformer中，多头注意力机制是一种机制，通过将输入进行多次不同的投影，以获得多个不同的注意力表示。每个注意力表示被称为一个头（head）。每个头都独立地学习不同的注意力模式，并捕捉输入中的不同关系和特征。

在多头注意力机制中，模型头通常指的是每个注意力头后面的投影层和相关参数。每个注意力头通过线性投影将输入进行变换，然后进行注意力计算。这些变换和计算是独立的，因此每个头可以学习不同的表示。最后，多个头的结果会被拼接在一起，并通过进一步的线性变换来生成最终的输出。

因此，多头注意力机制中的多个头和模型头指的是同一个概念，表示通过将输入进行多次投影并进行独立的注意力计算所得到的不同注意力表示。

Transformers中有许多不同的体系结构，每种体系结构都是围绕处理特定任务而设计的。以下是一个非详尽的列表：

    *Model (retrieve the hidden states)
    *ForCausalLM
    *ForMaskedLM
    *ForMultipleChoice
    *ForQuestionAnswering
    *ForSequenceClassification
    *ForTokenClassification

>在Hugging Face的Transformers库中，AutoModel类是一个通用的模型类，用于加载预训练的Transformer模型，但它没有特定的头部（task-specific head）。这意味着AutoModel类加载的模型只包含了Transformer的基本结构和参数，而没有针对特定任务的分类或回归头部。

>相比之下，AutoModelForSequenceClassification类是AutoModel的一个子类，专门用于序列分类任务。它在加载预训练的Transformer模型的基础上，还包括了一个特定的序列分类头部。这个头部是通过一些额外的层和参数来实现，用于将模型的输出映射到分类标签上，以便进行分类任务的训练和预测。

>因此，当我们需要进行序列分类任务时，例如将句子分类为肯定或否定，我们更倾向于使用AutoModelForSequenceClassification类，因为它已经包含了针对序列分类任务的头部，可以更方便地进行模型训练和预测。而AutoModel类则更适合于那些不需要特定任务头部的通用模型加载和应用场景。



In [ ]:
from transformers import AutoModelForSequenceClassification

checkpoint="distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs=model(**inputs)

现在，如果我们观察输出的形状，维度将低得多：模型头将我们之前看到的高维向量作为输入，并输出包含两个值的向量（每个标签一个）：

In [ ]:
print(outputs.logits.shape)

torch.Size([2, 2])


因为我们只有两个句子和两个标签，所以我们从模型中得到的结果是2 x 2的形状。

## 对输出进行后处理

我们从模型中得到的输出值本身并不一定有意义。我们来看看,

In [ ]:
print(outputs.logits)

tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)


我们的模型预测第一句为[-1.5607, 1.6123]，第二句为[ 4.1692, -3.3464]。这些不是概率，而是logits，即模型最后一层输出的原始非标准化分数。要转换为概率，它们需要经过SoftMax层（所有Transformers模型输出logits，因为用于训练的损耗函数通常会将最后的激活函数（如SoftMax）与实际损耗函数（如交叉熵）融合）：

In [ ]:
import torch

predictions=torch.nn.functional.softmax(outputs.logits,dim=-1)
print(predictions)

tensor([[4.0195e-02, 9.5980e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)


现在我们可以看到，模型预测第一句为[0.0402, 0.9598]，第二句为[0.9995, 0.0005]。这些是可识别的概率分数。

为了获得每个位置对应的标签，我们可以检查模型配置的id2label属性（下一节将对此进行详细介绍）：

In [ ]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

现在我们可以得出结论，该模型预测了以下几点：

* 第一句：否定：0.0402，肯定：0.9598
* 第二句：否定：0.9995，肯定：0.0005

我们已经成功地复制了管道的三个步骤：使用标记化器进行预处理、通过模型传递输入以及后处理！现在，让我们花一些时间深入了解这些步骤中的每一步。

## 总结

尝试用中文数据做一个总结。

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
from transformers import AutoTokenizer

checkpoint="papluca/xlm-roberta-base-language-detection"
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

raw_inputs=[
    "我怎么这么差劲。",
    "加油吧，世界上唯一的我！"
]
inputs=tokenizer(raw_inputs,padding=True,truncation=True,return_tensors="pt")
print(inputs)

In [3]:
from transformers import AutoModel

checkpoint="papluca/xlm-roberta-base-language-detection"
model=AutoModel.from_pretrained(checkpoint)

outputs=model(**inputs)
print(outputs.last_hidden_state.shape)

Some weights of the model checkpoint at papluca/xlm-roberta-base-language-detection were not used when initializing XLMRobertaModel: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaModel were not initialized from the model checkpoint at papluca/xlm-roberta-base-language-detection and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use 

torch.Size([2, 10, 768])


In [6]:
from transformers import AutoModelForSequenceClassification

checkpoint="papluca/xlm-roberta-base-language-detection"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

outputs=model(**inputs)
print(outputs.logits.shape)

torch.Size([2, 20])


In [7]:
print(outputs.logits)

tensor([[ 5.6219e-01, -7.9773e-01, -2.1926e-01, -1.5801e-01, -5.2928e-01,
         -1.0346e+00, -2.0762e-01, -3.8265e-01, -1.0614e-01,  1.1458e-01,
         -9.0685e-01, -5.6344e-01, -3.6693e-01, -1.1172e-01, -2.0979e-01,
          5.8208e+00,  1.2784e-01,  3.8908e-01, -5.6510e-01, -3.9662e-01],
        [-2.6321e-01, -7.8304e-01, -1.4672e-01, -3.9535e-03, -3.2122e-01,
         -7.7507e-01, -3.5481e-01, -7.4233e-02, -3.4361e-01,  7.8525e-01,
         -1.0026e+00, -6.1629e-01, -1.4243e-01, -9.3912e-01, -3.7985e-01,
          7.5082e+00,  2.2138e-01,  2.7081e-01, -7.6744e-01, -7.1168e-01]],
       grad_fn=<AddmmBackward0>)


In [8]:
model.config.id2label

{0: 'ja',
 1: 'nl',
 2: 'ar',
 3: 'pl',
 4: 'de',
 5: 'it',
 6: 'pt',
 7: 'tr',
 8: 'es',
 9: 'hi',
 10: 'el',
 11: 'ur',
 12: 'bg',
 13: 'en',
 14: 'fr',
 15: 'zh',
 16: 'ru',
 17: 'th',
 18: 'sw',
 19: 'vi'}